In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)

In [4]:
block_size = 3
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


In [14]:

def cmp(s,dt,t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:.15s} | exact: {ex} | approx: {app} | maxdiff: {maxdiff:.8f}')

In [5]:
n_embd = 10
n_hidden = 200

g= torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

#Layer1
W1 = torch.randn((block_size * n_embd, n_hidden), generator=g) * (5/3)/((block_size * n_embd)**0.5)
b1 = torch.randn(n_hidden, generator=g) *0.1

#Layer2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

#BatchNorm parameters
bn_gain = torch.ones((1, n_hidden))*0.1+1.0
bn_bias = torch.zeros((1, n_hidden))*0.1

#Note I am inializating many of these parameters in non-standard ways
# because sometimes initalizing with e.g. all zeros could mask bugs in backpropagation

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
for p in parameters:
    p.requires_grad = True


In [6]:
batch_size = 32
n = batch_size

ix = torch.randint(0, Xtr.shape[0], (n,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]


In [7]:
# forward pass

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

#Linear layer 1
hprebn = embcat @ W1 + b1

#BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar+1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bn_gain * bnraw + bn_bias

#Non-linearity
h = torch.tanh(hpreact)

#Linear layer 2
logits = h @ W2 + b2
#cross-entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[torch.arange(n), Yb].mean()

# backward pass with pytorch
for p in parameters:
    p.grad = None

for t in [logprobs,probs,counts,counts_sum,counts_sum_inv,
          norm_logits,logit_maxes,logits,h,
          hpreact,bnraw,bnvar_inv,bnvar,bndiff,hprebn,bnmeani
          ,embcat,emb]:
    t.retain_grad()
loss.backward()
loss


tensor(3.8279, grad_fn=<NegBackward0>)

In [ ]:
counts.shape, counts_sum.shape

# so if we change 

(torch.Size([32, 27]), torch.Size([32, 1]))

In [43]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n),Yb]=-1.0/n

cmp('logprobs', dlogprobs, logprobs)

dprobs = dlogprobs * 1.0/probs # chain it * local deriviate, derivative of log(probs) is 1/probs
cmp('probs', dprobs, probs)

# Here comes the interesting tensor parts
# lets take one element of probs probs[b, j] = counts[b, j] * counts_sum_inv[b]
# take the deriviative with regards to counts_sum_inv[b]:
# d(probs[b, j]) / d(counts_sum_inv[b]) = counts[b, j]
# which means dL/dcounts_sum_inv[b] = sum_j( dL/dprobs[b, j] * counts[b, j] )
dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
print(f'Shape counts_sum_inv: {counts_sum_inv.shape}, dcounts_sum_inv: {dcounts_sum_inv.shape}, counts: {counts.shape}, dprobs: {dprobs.shape}')

dcounts_sum = dcounts_sum_inv * -counts_sum**-2
cmp('counts_sum', dcounts_sum, counts_sum)

dcounts = dprobs*counts_sum_inv
dcounts+=torch.ones_like(counts)*dcounts_sum
cmp('counts', dcounts, counts)
dnorm_logits = norm_logits.exp()*dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

dlogit_maxes = -dnorm_logits.sum(1,keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
dlogits = dnorm_logits.clone()

cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs | exact: True | approx: True | maxdiff: 0.00000000
probs | exact: True | approx: True | maxdiff: 0.00000000
counts_sum_inv | exact: True | approx: True | maxdiff: 0.00000000
Shape counts_sum_inv: torch.Size([32, 1]), dcounts_sum_inv: torch.Size([32, 1]), counts: torch.Size([32, 27]), dprobs: torch.Size([32, 27])
counts_sum | exact: True | approx: True | maxdiff: 0.00000000
counts | exact: True | approx: True | maxdiff: 0.00000000
norm_logits | exact: True | approx: True | maxdiff: 0.00000000
logit_maxes | exact: True | approx: True | maxdiff: 0.00000000
logits | exact: False | approx: True | maxdiff: 0.00000001
